# <font color='blue'> CUSTOMER SEGMENTATION

In [1]:
#download necessary libraries

import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sqlite3

# connect to sqlite - make sure you have download the database from here into your data folder
# and you have created your notebooks in your notebooks folder
# otherwise adjust the file path accordingly

connect = sqlite3.connect("../data/bootcamp_db")

In [4]:
connect

In [5]:
#check the table
query= """
select *
from online_transactions_fixed ; 

"""
otc=pd.read_sql(query,connect)


In [6]:
otc.head(n=3)

,invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,total_order_value
0,536373,82494L,WOODEN FRAME ANTIQUE WHITE,6,2010-12-01T09:02:00,2.55,u1785,United Kingdom,15.30
1,536381,37444A,YELLOW BREAKFAST CUP AND SAUCER,1,2010-12-01T09:41:00,2.95,u15311,United Kingdom,2.95
2,536381,22438,BALLOON ART MAKE YOUR OWN FLOWERS,1,2010-12-01T09:41:00,1.95,u15311,United Kingdom,1.95



## 1.RFM calculation

In customer segmentation, one common approach is to use RFM analysis.
- R= Recency
- F= Frequency
- M=monetory

**Recency (R):** Recency refers to how recently a customer has made a purchase. It measures the time elapsed since the customer's last purchase. Customers who have made a purchase more recently are generally considered **more engaged and potentially more valuable.**
A higher recency value means that customer hasn't made a purchase recently, which typically indicates lower engagement or activity.

**Frequency (F):** Frequency refers to how often a customer makes purchases. It measures the number of times a customer has made a purchase within a specific period. Customers who make purchases more frequently are typically **more loyal and engaged with the brand.**

**Monetary Value (M):** Monetary value refers to the total amount of money spent by a customer on purchases. It measures the total value of all transactions made by the customer. Customers who spend more money are often considered **more valuable to the business.**


In [7]:
#When was the first and last purchase?
query2="""
select min(invoice_date) as first_purchase,
max(invoice_date) as last_purchase
from online_transactions_fixed """

In [8]:
pd.read_sql(query2,connect)

,first_purchase,last_purchase
0,2010-12-01T08:26:00,2011-12-09T12:50:00


#### <font color='blue'> Recency:customers that purchased recently


In [9]:
#check the type of invoice_date
otc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399841 entries, 0 to 399840
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   invoice            399841 non-null  object 
 1   stock_code         399841 non-null  object 
 2   description        399841 non-null  object 
 3   quantity           399841 non-null  int64  
 4   invoice_date       399841 non-null  object 
 5   price              399841 non-null  float64
 6   customer_id        399841 non-null  object 
 7   country            399841 non-null  object 
 8   total_order_value  399841 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 27.5+ MB


In [10]:
#convert invoice_date type to datetime

otc['invoice_date'] = pd.to_datetime(otc['invoice_date'])

In [11]:
#Since this transaction was in 2011 take last purchase date as current date. 
##(you can add +1 day to avoid zero value in coming recency calculation)
current_date = otc['invoice_date'].max()
current_date

Timestamp('2011-12-09 12:50:00')

In [12]:
#calculate the date of the customer's last purchase time
recent_purchased_date=otc.groupby('customer_id')['invoice_date'].max()
print(recent_purchased_date)

customer_id
u12346   2011-01-18 10:17:00
u12347   2011-12-07 15:52:00
u12348   2011-09-25 13:13:00
u12349   2011-11-21 09:51:00
u1235    2011-02-02 16:01:00
                 ...        
u1828    2011-03-07 09:52:00
u18281   2011-06-12 10:53:00
u18282   2011-12-02 11:43:00
u18283   2011-12-06 12:02:00
u18287   2011-10-28 09:29:00
Name: invoice_date, Length: 4363, dtype: datetime64[ns]


In [13]:
#Calculate the difference between the current date and the date of the customer's last purchase
recency = (current_date - recent_purchased_date).dt.days
print(recency)

customer_id
u12346    325
u12347      1
u12348     74
u12349     18
u1235     309
         ... 
u1828     277
u18281    180
u18282      7
u18283      3
u18287     42
Name: invoice_date, Length: 4363, dtype: int64


In [14]:
recency.sort_values(ascending=True)

customer_id
u16794      0
u12985      0
u17757      0
u17754      0
u13026      0
         ... 
u13747    373
u16583    373
u17968    373
u17908    373
u18074    373
Name: invoice_date, Length: 4363, dtype: int64

#### <font color='blue'> Frequency:frequency of transactions based on unique stock codes 

In [15]:
frequency = otc.groupby('customer_id')['stock_code'].nunique().sort_values(ascending=False)
print(frequency)


customer_id
u14911    1792
u12748    1767
u17841    1330
u14096    1119
u14298     884
          ... 
u13703       1
u16454       1
u16462       1
u18133       1
u12346       1
Name: stock_code, Length: 4363, dtype: int64


#### <font color='blue'> Monetory : is sum up the total amount spent by each customer 


In [16]:
monetary = otc.groupby('customer_id')['total_order_value'].sum().sort_values(ascending=False)
print(monetary)

customer_id
u14646    278778.02
u18102    259657.30
u1745     189575.53
u14911    132893.24
u12415    123638.18
            ...    
u12666      -227.44
u16252      -295.09
u16742      -464.90
u15823      -811.86
u14213     -1192.20
Name: total_order_value, Length: 4363, dtype: float64


In [17]:
## Convert the Series to DataFrames

monetary_df = monetary.reset_index()
frequency_df = frequency.reset_index()
recency_df = recency.reset_index()

In [18]:
#merge the table with customer_id
rfm_table = pd.merge(monetary_df, frequency_df, on='customer_id')
rfm_table = pd.merge(rfm_table, recency_df, on='customer_id')
rfm_table.columns = ['Customer_ID', 'Monetary', 'Frequency', 'Recency']

In [19]:
print(rfm_table)

     Customer_ID   Monetary  Frequency  Recency
0         u14646  278778.02        700        1
1         u18102  259657.30        150        0
2          u1745  189575.53        124        7
3         u14911  132893.24       1792        0
4         u12415  123638.18        443       23
...          ...        ...        ...      ...
4358      u12666    -227.44          2      358
4359      u16252    -295.09         21      365
4360      u16742    -464.90          3       46
4361      u15823    -811.86         15      336
4362      u14213   -1192.20          5      371

[4363 rows x 4 columns]


In [20]:
rfm_table.head()

,Customer_ID,Monetary,Frequency,Recency
0,u14646,278778.02,700,1
1,u18102,259657.30,150,0
2,u1745,189575.53,124,7
3,u14911,132893.24,1792,0
4,u12415,123638.18,443,23


## 2.Normalization of values 
- *Normalization* is the process of scaling the values to a standard range, typically between 0 and 1, to ensure that all the variables contribute equally to the analysis.



### Option 1 : using MinMaxScaler
- we need to download necessary libraries **from sklearn.preprocessing import MinMaxScaler** 

In [21]:
# Assuming you have the rfm_table DataFrame containing RFM values
rfm_values = rfm_table.set_index('Customer_ID')
rfm_values.head()

,Monetary,Frequency,Recency
Customer_ID,,,
u14646,278778.02,700,1
u18102,259657.30,150,0
u1745,189575.53,124,7
u14911,132893.24,1792,0
u12415,123638.18,443,23


In [22]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

In [23]:
#Fit and transform the RFM values using the scaler
rfm_scaled = scaler.fit_transform(rfm_values)

In [24]:
#Convert the scaled values back into a DataFrame
rfm_normalized = pd.DataFrame(rfm_scaled, columns=['Normalized_Frequency', 'Normalized_Recency', 'Normalized_Monetary'])
print(rfm_normalized)

      Normalized_Frequency  Normalized_Recency  Normalized_Monetary
0                 1.000000            0.390285             0.002681
1                 0.931704            0.083194             0.000000
2                 0.681386            0.068677             0.018767
3                 0.478928            1.000000             0.000000
4                 0.445870            0.246790             0.061662
...                    ...                 ...                  ...
4358              0.003446            0.000558             0.959786
4359              0.003204            0.011167             0.978552
4360              0.002598            0.001117             0.123324
4361              0.001359            0.007817             0.900804
4362              0.000000            0.002233             0.994638

[4363 rows x 3 columns]


In [25]:
rfm_normalized.head()

,Normalized_Frequency,Normalized_Recency,Normalized_Monetary
0,1.000000,0.390285,0.002681
1,0.931704,0.083194,0.000000
2,0.681386,0.068677,0.018767
3,0.478928,1.000000,0.000000
4,0.445870,0.246790,0.061662


### Option 2 : Calculate the Range and Normalize the Range:
(referance:https://www.geeksforgeeks.org/rfm-analysis-analysis-using-python/)
- **Calculate the Range:** For each value in the 'Frequency' column, we find out how far it is from the smallest value in the column. This tells us how spread out the values are.
- **Normalize the Range:** After calculating the range, we want to scale the values so they all fit within the same range. We do this by dividing each value by the total range of values in the column. This ensures that no matter what the original range was, the values will always fall between 0 and 1.


In [26]:
#normalization for Monetary and Frequency

rfm_table['Normalized_Monetary'] = (rfm_table.Monetary - rfm_table.Monetary.min())/(rfm_table.Monetary.max() - rfm_table.Monetary.min())
rfm_table['Normalized_Frequency'] = (rfm_table.Frequency - rfm_table.Frequency.min())/(rfm_table.Frequency.max() - rfm_table.Frequency.min())

#higher recency values indicate less desirable behavior so we subtract each value from the maximum value and divide by the negative range. 

rfm_table['Normalized_Recency'] = (rfm_table.Recency - rfm_table.Recency.max())/(rfm_table.Recency.min() - rfm_table.Recency.max())

In [27]:
rfm_table.head()

,Customer_ID,Monetary,Frequency,Recency,Normalized_Monetary,Normalized_Frequency,Normalized_Recency
0,u14646,278778.02,700,1,1.000000,0.390285,0.997319
1,u18102,259657.30,150,0,0.931704,0.083194,1.000000
2,u1745,189575.53,124,7,0.681386,0.068677,0.981233
3,u14911,132893.24,1792,0,0.478928,1.000000,1.000000
4,u12415,123638.18,443,23,0.445870,0.246790,0.938338


## 3.Calculating RFM score
- We need to assign weights to the normalized rank values for Recency (R), Frequency (F), and Monetary (M).
Each weight represents the relative importance of that RFM component in determining the overall RFM score.
- Deciding the weights for each RFM component involves determining the relative importance of these components in the context of your business objectives and customer behavior.
For example,
   - if acquiring new customers is a top priority, you might assign a higher weight to Frequency to emphasize customer loyalty and retention
   - in subscription-based businesses, Recency might be more critical for retaining customers.
   - E-commerce retailers often track the monetary value of each customer's transactions to identify high-value customers, tailor marketing strategies, and optimize pricing and product offerings.
- For this project it is assumed that maximizing revenue and identifying high-value customers are primary goals so I assign a higher weight to Monetary

In [28]:
rfm_table['RFM_Score'] = 0.15*rfm_table['Normalized_Recency']+0.25 * \
    rfm_table['Normalized_Frequency']+0.6*rfm_table['Normalized_Monetary']
rfm_table = rfm_table.round(2)
rfm_table[['Customer_ID', 'RFM_Score']].head(10)

,Customer_ID,RFM_Score
0,u14646,0.85
1,u18102,0.73
2,u1745,0.57
3,u14911,0.69
4,u12415,0.47
5,u14156,0.49
6,u17511,0.41
7,u16684,0.31
8,u14096,0.45
9,u13694,0.34


In [29]:
rfm_table.describe()

,Monetary,Frequency,Recency,Normalized_Monetary,Normalized_Frequency,Normalized_Recency,RFM_Score
count,4363.000000,4363.000000,4363.000000,4363.000000,4363.000000,4363.000000,4363.000000
mean,1898.571497,61.179234,91.154710,0.012026,0.033681,0.755432,0.128357
std,8300.366043,85.371220,100.960982,0.029697,0.047773,0.270558,0.052619
min,-1192.200000,1.000000,0.000000,0.000000,0.000000,-0.000000,0.000000
25%,292.585000,15.000000,16.000000,0.010000,0.010000,0.620000,0.100000
50%,642.770000,35.000000,49.000000,0.010000,0.020000,0.870000,0.140000
75%,1584.030000,77.000000,142.000000,0.010000,0.040000,0.960000,0.160000
max,278778.020000,1792.000000,373.000000,1.000000,1.000000,1.000000,0.850000


## 4. Customer Segmentation
- adjust tresholds to create segments that match your business goals

In [30]:
def customer_segment(i):
    if i < 0.14:
        return 'low value'
    elif i>=0.14 and i<0.20:
        return 'mid value' 
    else:
        return 'high value'
        

In [31]:
rfm_table['customer_segment']=rfm_table['RFM_Score'].apply(customer_segment)

In [32]:
rfm_table

,Customer_ID,Monetary,Frequency,Recency,Normalized_Monetary,Normalized_Frequency,Normalized_Recency,RFM_Score,customer_segment
0,u14646,278778.02,700,1,1.00,0.39,1.00,0.85,high value
1,u18102,259657.30,150,0,0.93,0.08,1.00,0.73,high value
2,u1745,189575.53,124,7,0.68,0.07,0.98,0.57,high value
3,u14911,132893.24,1792,0,0.48,1.00,1.00,0.69,high value
4,u12415,123638.18,443,23,0.45,0.25,0.94,0.47,high value
...,...,...,...,...,...,...,...,...,...
4358,u12666,-227.44,2,358,0.00,0.00,0.04,0.01,low value
4359,u16252,-295.09,21,365,0.00,0.01,0.02,0.01,low value
4360,u16742,-464.90,3,46,0.00,0.00,0.88,0.13,low value
4361,u15823,-811.86,15,336,0.00,0.01,0.10,0.02,low value


In [33]:
#check how many customers are in each segment
rfm_table.customer_segment.value_counts()

customer_segment
mid value     2317
low value     1894
high value     152
Name: count, dtype: int64

In [34]:
#Define the 'High Value' segment based on your segmentation criteria

rfm_table[rfm_table.customer_segment=='high value'].describe()

,Monetary,Frequency,Recency,Normalized_Monetary,Normalized_Frequency,Normalized_Recency,RFM_Score
count,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000
mean,21768.690592,319.440789,6.894737,0.081711,0.177697,0.981184,0.241053
std,38700.905765,253.299753,8.358806,0.138367,0.141424,0.022225,0.096400
min,1070.470000,9.000000,0.000000,0.010000,0.000000,0.880000,0.200000
25%,5211.177500,208.750000,1.000000,0.020000,0.120000,0.970000,0.200000
50%,9651.495000,287.000000,3.000000,0.040000,0.160000,0.990000,0.210000
75%,19753.467500,365.500000,10.000000,0.072500,0.200000,1.000000,0.230000
max,278778.020000,1792.000000,46.000000,1.000000,1.000000,1.000000,0.850000
